In [1]:
#To run the animations in a separate window, use the line below.
%matplotlib tk
#More information on running animations through Jupyter Notebook can be found on this Stack Exchange: https://stackoverflow.com/questions/25333732/matplotlib-animation-not-working-in-ipython-notebook-blank-plot
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
plt.rcParams["figure.figsize"] = (20, 20) #Changes the default size of plotted figures.
import random as rand
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation


Let's try to make the simulation 3 dimensional

In [2]:
#G = 6.674 * 10 ** (-11) * (3600 * 24) ** 2 #m^3 kg^-1 day^-2
G = 6.674 * 10 ** (-11) * (3600 ** 2) #m^3 kg^-1 hr^-2
#G = 6.674 * 10 ** (-11) ##m^3 kg^-1 s^-2


#Standard Distance Equation
def calc_distance(x_position_1 = 0, y_position_1 = 0, z_position_1 = 0, x_position_2 = 0, y_position_2 = 0, z_position_2 = 0): 
    R = np.sqrt((x_position_1 - x_position_2)**2 + (y_position_1 - y_position_2)**2 + (z_position_1 - z_position_2) ** 2)
    return R
#Newtonian equations of motion for velocity and position.
def update_velocity(velocity, acceleration, timestep):
    new_velocity = velocity + (acceleration * timestep)
    return new_velocity      
def update_position(x, y, z, v_x, v_y, v_z, timestep):
    new_x = x + v_x*timestep
    new_y = y + v_y*timestep
    new_z = z + v_z*timestep
    return new_x, new_y, new_z
def calculate_accelerations(bodies):
    x_accelerations = []
    y_accelerations = []
    z_accelerations = []
    for orbitting_body in bodies:
        x_accel = 0
        y_accel = 0
        z_accel = 0
        for central_body in bodies:
            if central_body != orbitting_body:
                R = calc_distance(central_body.x_position, central_body.y_position, central_body.z_position, orbitting_body.x_position, orbitting_body.y_position, orbitting_body.z_position)
                x_accel += -G * central_body.mass * np.abs(R ** (-3)) * (-central_body.x_position + orbitting_body.x_position)
                y_accel += -G * central_body.mass * np.abs(R ** (-3)) * (-central_body.y_position + orbitting_body.y_position)
                z_accel += -G * central_body.mass * np.abs(R ** (-3)) * (-central_body.z_position + orbitting_body.z_position)
        x_accelerations.append(x_accel)
        y_accelerations.append(y_accel)
        z_accelerations.append(z_accel)

    return x_accelerations, y_accelerations, z_accelerations

#The heart of the simulation.


def calculate_step(bodies, t=1):
    for i in range(0, len(bodies)):
        bodies[i].x_position, bodies[i].y_position, bodies[i].z_position = update_position(bodies[i].x_position, bodies[i].y_position, bodies[i].z_position, bodies[i].x_velocity, bodies[i].y_velocity, bodies[i].z_velocity, t)
    x_accelerations, y_accelerations, z_accelerations = calculate_accelerations(bodies)
    for i in range(0, len(bodies)):
        bodies[i].x_velocity = update_velocity(bodies[i].x_velocity, x_accelerations[i], t)
        bodies[i].y_velocity = update_velocity(bodies[i].y_velocity, y_accelerations[i], t)
        bodies[i].z_velocity = update_velocity(bodies[i].z_velocity, z_accelerations[i], t)
    return
        
#generate randomized initial conditions
def generate_initial(bodies):
    for body in bodies:
        time = rand.randint(0, 100)
        print(time)
        i = 0
        while i < time:
            calculate_step([body])
            i+=1
    return 

#Simulation Wrapper: This automates what I did manually in previous works. This code can now take N bodies and handle them
#Computing power allowing, without requiring the user to initialize all of the arrays and appending them.
def simulate(bodies, simulation_time, lim_low = -5 * 10 ** 12, lim_high = 5 * 10 ** 12):
    file = open("simulation.py", "w")
    for body in bodies:
        file.write(body.name + "_x_position = [] \n")
        file.write(body.name + "_y_position = [] \n")
        file.write(body.name + "_z_position = [] \n")
    file.write("for t in range(0, simulation_time): \n")
    for body in bodies:
        file.write("  " + body.name + "_x_position.append(" + body.name + ".x_position) \n")
        file.write("  " + body.name + "_y_position.append(" + body.name + ".y_position) \n")
        file.write("  " + body.name + "_z_position.append(" + body.name + ".z_position) \n")
    file.write("  calculate_step([")
    file.write(bodies[0].name)
    for body in bodies[1:]:
        file.write(", " + body.name)
    file.write("]) \n")

    #Set plot and color configurations
    #Information for changing default color cycle: https://stackoverflow.com/questions/44806598/matplotlib-set-color-cycle-versus-set-prop-cycle
    #https://stackoverflow.com/questions/16006572/plotting-different-colors-in-matplotlib
    file.write("fig = plt.figure() \n")
    file.write("ax = fig.gca(projection = \'3d\')\n")
    file.write("colors = [plt.cm.get_cmap(\'cool\')(i) for i in np.linspace(0, 1, " + str(len(bodies)) + ")]\n")
    file.write("ax.set_prop_cycle('color', colors)\n")
    
    
    for body in bodies:
        file.write("ax.plot(" + body.name + "_x_position, " + body.name + "_y_position, " + body.name + "_z_position, label = \'" + str(body.name) + "\')\n")
    file.write("plt.legend() \n")
    
    
    #Mandate cubic field of view. Make the field of view large enough that the plots are not scrunched together.
    file.write("ax.set_xlim3d(" + str(lim_low) + ", " + str(lim_high) + ")\n")
    file.write("ax.set_ylim3d(" + str(lim_low) + ", " + str(lim_high) + ")\n")
    file.write("ax.set_zlim3d(" + str(lim_low) + ", " + str(lim_high) + ")\n")
    file.write("ax.set_facecolor(\'black\')\n")
    file.write("plt.show() \n")
    file.close()
    exec(open("simulation.py").read()) #Information for running programs within Python programs can be found here: https://stackoverflow.com/questions/7974849/how-can-i-make-one-python-file-run-another
    return
    

    
class body:
    global G
    def __init__(self, name, mass, x_position, y_position, z_position, x_velocity = 0.0, y_velocity = 0.0, z_velocity = 0.0):
        self.name = name
        self.mass = mass
        self.x_position = x_position
        self.y_position = y_position
        self.z_position = z_position
        self.x_velocity = x_velocity
        self.y_velocity = y_velocity
        self.z_velocity = z_velocity

        


In [3]:
#Initial Conditions Values taken from NASA's fact sheets on solar system bodies
#https://nssdc.gsfc.nasa.gov/planetary/planetfact.html

M1 = 1.9091 * 10 ** 30 #kg #solar mass
M2 = 5.9724 * 10 ** 24 #kg #earth mass
M3 = 6.4171 * 10 ** 23 #kg #mars mass


v_earth = (30.29 * 10 ** 3) * 3600# m/hr max speed
x_earth = 147.09 * 10 ** 9 #m at perihelion
y_earth = 0.0 #m
z_earth = 0.0

v_mars = (26.5 * 10 ** 3) * 3600 #m/hr max speed
x_mars = 206.62 * 10 ** 9 #m at perihelion
y_mars = 0.0
z_mars = 0.0


Sun = body("Sun", M1, 0.0, 0.0, 0.0, 0.0, 0.0)
Earth = body("Earth", M2, x_earth, y_earth, z_earth, 0.0, v_earth)
Mars = body("Mars", M3, x_mars, y_mars, z_mars, 0.0, v_mars)

print(Sun.name)
bodies = [Sun, Earth, Mars]

Sun


In [4]:
Sun_x_position = []
Sun_y_position = []
Earth_x_position = []
Earth_y_position = []

for i in range(0, 6000):
    Sun_x_position.append(Sun.x_position)
    Sun_y_position.append(Sun.y_position)
    Earth_x_position.append(Earth.x_position)
    Earth_y_position.append(Earth.y_position)
    calculate_step(bodies)
    
plt.plot(Sun_x_position, Sun_y_position)
plt.plot(Earth_x_position, Earth_y_position)
plt.show()

In [13]:
#Matplotlib animation information can be found on their online documentation: https://matplotlib.org/api/_as_gen/matplotlib.animation.Animation.html#matplotlib.animation.Animation
Sun = body("Sun", M1, 0.0, 0.0, 0.0, 0.0, 0.0)
Earth = body("Earth", M2, x_earth, y_earth, z_earth, 0.0, v_earth)
Mars = body("Mars", M3, x_mars, y_mars, z_mars, 0.0, v_mars)
bodies = [Sun, Earth, Mars]

Sun_x_position = []
Sun_y_position = []
Earth_x_position = []
Earth_y_position = []
Mars_x_position = []
Mars_y_position = []

for i in range(0, 6000):
    Sun_x_position.append(Sun.x_position)
    Sun_y_position.append(Sun.y_position)
    Earth_x_position.append(Earth.x_position)
    Earth_y_position.append(Earth.y_position)
    Mars_x_position.append(Mars.x_position)
    Mars_y_position.append(Mars.y_position)
    calculate_step(bodies)
    
print(Earth_x_position[100], Earth_y_position[100])
fig = plt.figure()
ax = fig.add_subplot(111)



sun_plot, = ax.plot(Sun.x_position, Sun.y_position, 'y*', markersize = 20, label = "Sun")
sun_trail, = ax.plot(Sun.x_position, Sun.y_position, 'y-')
earth_plot, = ax.plot(Earth.x_position, Earth.y_position, 'bo', markersize = 10, label = "Earth")
earth_trail, = ax.plot(Earth.x_position, Earth.y_position, 'b-')
mars_plot, = ax.plot(Mars.x_position, Mars.y_position, 'ro', markersize = 10, label = "Earth")
mars_trail, = ax.plot(Mars.x_position, Mars.y_position, 'r-')
ax.set_xlim(-1 * 10 ** 12, 1 * 10 ** 12)
ax.set_ylim(-1 * 10 ** 12, 1 * 10 ** 12)
N = 100
def update(frame_number):
    #for i in range(0, frame_number):
        #Sun_x_position.append(Sun.x_position)
        #Sun_y_position.append(Sun.y_position)
        #Earth_x_position.append(Earth.x_position)
        #Earth_y_position.append(Earth.y_position)
        #Mars_x_position.append(Mars.x_position)
        #Mars_y_position.append(Mars.y_position)
        #calculate_step(bodies)
    #ax.plot(Sun_x_position, Sun_y_position, 'y*', markersize = 45, label = "Sun")
    #ax.plot(Earth_x_position[:-1], Earth_y_position[:-1], markersize = 10, label = "Earth")
    #ax.plot(Mars_x_position[:-1], Mars_y_position[:-1], markersize = 10, label = "Earth")
    #sun_plot.set_data(Sun_x_position[-N:], Sun_y_position[-N:])
    #earth_plot.set_data(Earth_x_position[-N:], Earth_y_position[-N:])
    #mars_plot.set_data(Mars_x_position[-N:], Mars_y_position[-N:])
    sun_plot.set_data(Sun_x_position[frame_number], Sun_y_position[frame_number])
    sun_trail.set_data(Sun_x_position[:frame_number], Sun_y_position[:frame_number])
    earth_plot.set_data(Earth_x_position[frame_number], Earth_y_position[frame_number])
    earth_trail.set_data(Earth_x_position[:frame_number], Earth_y_position[:frame_number])
    mars_plot.set_data(Mars_x_position[frame_number], Mars_y_position[frame_number])
    mars_trail.set_data(Mars_x_position[:frame_number], Mars_y_position[:frame_number])
    return sun_plot, earth_plot, mars_plot, sun_trail, earth_trail, mars_trail,

animation_bod2 = animation.FuncAnimation(fig, update, interval = 90)
plt.show()
print(Earth_x_position[10:N], Earth_y_position[10:N])

146712404327.2205 10894974848.897322
[147086565512.42584, 147085802298.39297, 147084962765.09818, 147084046913.11728, 147083054743.07278, 147081986255.6338, 147080841451.51602, 147079620331.48184, 147078322896.3402, 147076949146.94675, 147075499084.20367, 147073972709.0598, 147072370022.51062, 147070691025.59818, 147068935719.41116, 147067104105.0849, 147065196183.8013, 147063211956.7889, 147061151425.32285, 147059014590.72488, 147056801454.36337, 147054512017.65326, 147052146282.05612, 147049704249.0801, 147047185920.28, 147044591297.25717, 147041920381.65958, 147039173175.1818, 147036349679.56494, 147033449896.59677, 147030473828.11163, 147027421475.99042, 147024292842.1606, 147021087928.5963, 147017806737.31815, 147014449270.39337, 147011015529.9358, 147007505518.10577, 147003919237.1103, 147000256689.20282, 146996517876.68347, 146992702801.89886, 146988811467.2422, 146984843875.1532, 146980800028.11823, 146976679928.67007, 146972483579.38818, 146968210982.89847, 146963862141.87344,

In [7]:

fig, ax = plt.subplots()

x = np.arange(0, 2*np.pi, 0.01)
line, = ax.plot(x, np.sin(x))


def animate(i):
    line.set_ydata(np.sin(x + i / 50))  # update the data.
    return line,


ani = animation.FuncAnimation(
    fig, animate, interval=20, blit=True, save_count=50)
plt.show()